### 사이킷 런 학습
1. Cross Validation 이해
    1. K-fold
    1. Stratified-fold
1. 모델에서 Dateset을 나누는 이유
    

### Cross Validation의 이해

#### Dataset?
    
    * Machine Learning 모델을 만들기 위해 필요한 데이터 모임
    * Machine Learning을 통해 해석이 필요한 데이터 모임
    Vision에서 MINST, CIFAR-10, ImageNet 데이터 셋을 자주사용
    
#### Dateset나누기
![Dataset](https://user-images.githubusercontent.com/50629716/61774441-d1eb4c80-ae31-11e9-82c4-d6a9f8058467.JPG)

#### Train model
    
   * 모델의 Hyper-Parameter을 올바르게 정해주어야 한다
        * lr, batch size 등등
   * Bias : 모델의 Training data에 대한 예측값과 실제값의 오차
   * Variance : 모델의 test data에 대한 예측값과 실제값의 오차
    
   ![bias](https://user-images.githubusercontent.com/50629716/61774729-73729e00-ae32-11e9-9bd0-c2101b12cf76.JPG)
   
   * High Bias : 훈련데이터를 제대로 표현 못함
   * High variance : 훈련데이터를 지나치게 자세히 표현함. 새로운 데이터 예측 불가
   
   * Validation : train셋으로 학습한 모델이 'Sample Bias' 되지 않았는지 확인
       * Sample Bias : 훈련에 사용하지 않은 데이터에서 낮은 정확도를 보이는
   
   * 데이터 셋이 작으면 train/valid/test로 분류하기 어려움
       * 이럴땐 Cross Validation사용
            * Train/test 두 개로 나눔
            * 작은 데이터 셋에도 좋은 정확도
            * k-Fold / Stratified K-Fold/ Leave one out/ Shuffle-split/ Bootstrap
            

### Cross Validation 종류

#### K-Fold

K개의 Fold를 만들어 진행하는 교차 검증식
K개의 모델의 Hyper-Parameter의 평균을 최종 결과로 사용

    * 데이터 갯수가 적은 데이터 셋에서 정확도 향상
    * 기존의 Train과 test로만 분류해 데이터 셋의 양이 더 많다
    
EX) 데이터가 150개 일때 K를 10으로 정하면 135개 Train, 15개 Test 
    10번 씩 135개 15개 나누어서 데이터 학습 시킴

In [100]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

iris = load_iris()

kfold = KFold(n_splits = 3)
# 분류 모델 불러오기
dt_clf = DecisionTreeClassifier()
# data를 k만큼 나눈다 k는 Hyper Parameter
for train_idx, test_idx in kfold.split(iris.data):
        X_train, X_test = iris.data[train_idx], iris.data[test_idx]
        y_train, y_test = iris.target[train_idx], iris.target[test_idx]
        print(train_idx,test_idx)
        dt_clf.fit(X_train,y_train)
        pred = dt_clf.predict(X_test)
        accuracy = np.round(accuracy_score(y_test,pred),4)
        # 정확도 출력
        print(accuracy*100)

[ 50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85
  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103
 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121
 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 146 147 148 149] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
0.0
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49 100 101 102 103
 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121
 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 146 147 148 149] [50 51 52 53 54 55 56 57 58 

### Stratified K-Fold

데이터의 분포가 불균형할 때 데이터의 비율에 맞게 데이터 세트를 나누는 K-fold 방식
데이터가 편향되어 있을 때(데이터가 몰려있을때) 사용

StratifiedKFold(n_split, shuffle, random_state)
n_split은 몇개로 분할할지
Shuffle은 섞일지 않섞을지(True, False)

K-FOld랑 차이를 보면은 인덱스 값들이 좀더 넓게 분포되어 있음고로 데이터가 몰려있는일을 방지가능


In [97]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
iris = load_iris()

# 데이터 분포를 확인하기 위해 DataFrame을 만들어줍니다.
iris_df = pd.DataFrame(data = iris.data, columns = iris.feature_names)
iris_df['label'] = iris.target

# Classification에 사용할 모델 불러오기
dt_clf = DecisionTreeClassifier()

# Iris 데이터는 3개의 Class로 이루어져 있습니다.
# 2. 그러므로 3개의 Fold로 데이터를 나눕니다.
n_iter = 0
skf = StratifiedKFold(n_splits = 4)
avg_acc = []

for train_idx, test_idx in skf.split(iris.data, iris.target) :    # iris 데이터에서 나누기
    # Iter 수 증가
    n_iter += 1
    
    # 3. K 개수 만큼 Fold 나누기
    train_label = iris_df['label'].iloc[train_idx]      
    test_label = iris_df['label'].iloc[test_idx]
    print(train_idx),test_idx)
    X_train, X_test = iris.data[train_idx], iris.data[test_idx]
    y_train, y_test = iris.target[train_idx], iris.target[test_idx]
    
    print("Iteration :", n_iter)
    print("--------------------")
    print("학습 데이터 분포 : \n", train_label.value_counts())
    print("--------------------")
    print("검증 데이터 분포 : \n", test_label.value_counts())
    print("--------------------")
    
    

111 39
Iteration : 1
--------------------
학습 데이터 분포 : 
 2    37
1    37
0    37
Name: label, dtype: int64
--------------------
검증 데이터 분포 : 
 2    13
1    13
0    13
Name: label, dtype: int64
--------------------
111 39
Iteration : 2
--------------------
학습 데이터 분포 : 
 2    37
1    37
0    37
Name: label, dtype: int64
--------------------
검증 데이터 분포 : 
 2    13
1    13
0    13
Name: label, dtype: int64
--------------------
114 36
Iteration : 3
--------------------
학습 데이터 분포 : 
 2    38
1    38
0    38
Name: label, dtype: int64
--------------------
검증 데이터 분포 : 
 2    12
1    12
0    12
Name: label, dtype: int64
--------------------
114 36
Iteration : 4
--------------------
학습 데이터 분포 : 
 2    38
1    38
0    38
Name: label, dtype: int64
--------------------
검증 데이터 분포 : 
 2    12
1    12
0    12
Name: label, dtype: int64
--------------------


사이킷런 익히기
sklearn.datasets : 예제 데이터 세트 제공함
### 여기선 Iris 데이터 셋을 제공
ML 알고리즘
sklearn.ensemble : 앙상블 알고리즘 제공
sklearn.linear_model : 선형 회귀 및 로지스틱 회귀 등 Regression 관련 알고리즘 지원
sklearn.naive_bayes : 나이브 베이즈 알고리즘 제공
sklearn.neighbors : 최근접 이웃 알고리즘 제공
sklearn.svm : Support Vector Machine 알고리즘 제공
sklearn.tree : 의사 결정 트리 알고리즘 제공
sklearn.cluster : 비지도 클러스터링 (Unsupervised Clustering) 알고리즘 제공
각종 모델을 사용

사용할 모델.fit(train데이터, train종류들)
pred = 사용할 모델.predict(train데이터)

정확도 검증 : sklearn.metrics(train종류들, pred)

In [16]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
data_clf = DecisionTreeClassifier()
data_clf.fit(iris.data,iris.target)
pred = data_clf.predict(iris.data)
print(accuracy_score(iris.target,pred)*100)
#위와같은 정확도는 전체데이터를 검증데이터로 사용하므로 정확도가 모두같은 100% 확률이 나온다

X_train, X_test, Y_train, Y_test = train_test_split(iris.data,pred,test_size = 0.25) 
data_clf.fit(X_train, Y_train) #데이터들을 data_clf모델로 학습 진행
pred = data_clf.predict(X_test) #위의 학습한 데이터를 성능 평가를위해 pred리스트에 삽입

print(accuracy_score(Y_test,pred))

100.0
0.9210526315789473


train_test_split(arrays, test_size, train_size, random_state, shuffle, stratify)
arrays : 데이터 입력
test_size : Test data의 비율 (default = 0.25)
train_size : Train data의 비율 (default = 나머지 데이터)
random_state : shuffle을 위한 시드값
shuffle : shuffle 여부 (default = True)
stratify : Dataset의 비율 유지